In [1]:
# from dataraw_sampling_SETShatter import *
from collections import defaultdict, Counter
import itertools
import copy
import random
import json
import operator as op
from functools import reduce
from tqdm import tqdm
from itertools import chain, combinations
import numpy as np
import time

In [4]:
# Resolve one cardpair

def resolve_proppair(prop1, prop2, all_vals_tuple, all_vals_set):
    if prop1 == len(all_vals_tuple) or prop2 == len(all_vals_tuple): # '*' symbol
        # *
        return all_vals_tuple
    elif prop1 == prop2:
        return (prop1,)
    else:
        return tuple(all_vals_set - set([prop1, prop2]))

def resolve_cardpair(card1_prop, card2_prop, all_vals_list, all_vals_set):
    '''
    card1_prop/card2_prop: [1, 0 ,*], [2,0,1]
    return answer cards
    '''
    ind_ans = [resolve_proppair(prop1, prop2, all_vals_list, all_vals_set) for prop1, prop2 in zip(card1_prop, card2_prop)]
    return set(itertools.product(*ind_ans))


def draw_cardpair(num_attributes, num_attr_vals):
    all_vals_tuple = tuple(range(num_attr_vals))
    all_vals_set = set(all_vals_tuple)
    
    card1_prop = np.random.choice(a=num_attr_vals, size=num_attributes, replace=True)
    card2_prop = np.random.choice(a=num_attr_vals+1, size=num_attributes, replace=True)
    subset = resolve_cardpair(card1_prop, card2_prop, all_vals_tuple, all_vals_set)
    # list of tuples, each one card
    return subset

In [22]:
set_fns = {
    '|': lambda x,y: x | y,
    '&': lambda x,y: x & y,
    '-': lambda x,y: x - y,
    '!': lambda x,y: x.symmetric_difference(y)
}

def draw_two_cardpairs(num_attributes, num_attr_vals):
    all_subsets = []
    subset1 = draw_cardpair(num_attributes, num_attr_vals)
    subset2 = draw_cardpair(num_attributes, num_attr_vals)
    op = np.random.choice(a=('|', '&', '-', '!'))
#     print(subset1, op, subset2)
    ans_subset = set_fns[op](subset1, subset2)
    return ans_subset, len(ans_subset)

In [33]:
all_ans_subsets = set()
all_ans_subsets_count = defaultdict(int)
ans_subset_sizes = []

for _ in range(10000):
    # set, int
    ans_subset, ans_subset_size = draw_two_cardpairs(num_attributes=4, num_attr_vals=3)
    ans_subset = tuple(sorted(ans_subset))
    all_ans_subsets.add(ans_subset)
    all_ans_subsets_count[ans_subset] += 1
    
print('Number of card subsets generated =', len(all_ans_subsets))

Number of card subsets generated = 4918


In [28]:
# 4-3
# draw 1000, return 679 
# draw 10000, return 4960 
# draw 100000, return 26122

In [27]:
26122 / 100000

0.26122

In [29]:
np.log2(26122)

14.67297773890013

In [2]:
def ncr(n, r):
    r = min(r, n-r)
    numer = reduce(op.mul, range(n, n-r, -1), 1)
    denom = reduce(op.mul, range(1, r+1), 1)
    return numer // denom

In [3]:
1 + 1 + ncr(5, 2) + ncr(5,3) + ncr(5,4)

27

In [4]:
243 * 27

6561

In [5]:
1 + 1 + ncr(6, 2) + ncr(6,3) + ncr(6,4) + ncr(6,5)

58

In [7]:
3**6 * 58

42282